In [4]:
import pyiast
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Aiida Lab data importation

In [5]:
#entre liste des pression et liste des loading_absolute_average, les transfo en csv file puis les importe sur ce doc
def conv_to_csv(pressure,loading_absolute_average,mof_gas):
    df = pd.DataFrame({
        'Pressure (bar)': pressure,
        'Loading (mmol/g)': loading_absolute_average
    })

    # Save data into a csv file
    df.to_csv('isotherm_data_{mof_gas}.csv', index=False)
#    return

    #import data from the csv file
#def import_data(mof_gas):
    df_mof = pd.read_csv("isotherm_data_{mof_gas}.csv")
    return df_mof

#### Loading q values in pure conditions

In [6]:
def get_pure_q(df_mof):
    pure_isotherm = pyiast.InterpolatorIsotherm(df_mof,
                                        loading_key="Loading (mmol/g)",
                                        pressure_key="Pressure (bar)")
    pure_isotherm = pyiast.InterpolatorIsotherm(df_mof,
                                        loading_key="Loading (mmol/g)",
                                        pressure_key="Pressure (bar)",
                                        fill_value=df_mof['Loading (mmol/g)'].max())
    return pure_isotherm

#### Loading q values in binary mixture

In [7]:
def get_mixt_q(N2_isotherm, CO2_isotherm, pressure):
    p_part = pressure*np.array([0.85,0.15])
    q_mix = pyiast.iast(p_part, [N2_isotherm, CO2_isotherm], verboseflag=False)
    return q_mix

#### selectivity and q_mix calculation

In [10]:
def get_selectivity(mof,pressures,loading_absolute_average, S_pressure):
    name_N2 = f"{mof}_N2"
    name_CO2 = f"{mof}_CO2"
    
    df_N2 = conv_to_csv(pressures, loading_absolute_average, name_N2)
    N2_isotherm = get_pure_q(df_N2)

    df_CO2 = conv_to_csv(pressures, loading_absolute_average, name_CO2)
    CO2_isotherm = get_pure_q(df_CO2)
    
    q_mix = get_mixt_q(N2_isotherm, CO2_isotherm, S_pressure)
    Selectivity_mof = (0.85/0.15)*(q_mix[1]/q_mix[0])
    return Selectivity_mof , q_mix

In [13]:
#entrer vakeurs de Aiida lab
P_list = [0.2, 0.8, 1.4, 2.0, 2.6, 3.2, 3.8, 4.4, 5.0, 5.6, 6.2, 6.8, 7.4, 8.0, 8.6, 9.2, 9.8, 10.2]
q_mean=[0.23575968759626, 0.98475549405786, 1.8503006720399, 2.7077428446667, 3.6901292397011, 4.821734538441, 5.8649964779462, 6.635211171313, 7.4943700823624, 8.2962757670132, 8.5992285964759, 9.203984040658, 9.7064218748443, 10.027881144504, 10.348482044953, 10.553271771403, 10.764379095248, 10.953048647908]

#donner nom du mof
mof = "Mg_74_CO2"

S_pressure = 1
S , q_mix = get_selectivity(mof,P_list,q_mean, S_pressure)

print(S)
print(q_mix) #q de N2 puis q de CO2

1.0000000000000002
[1.08227997 0.19099058]
